<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EA%B0%81%EC%B8%B5_%EC%88%9C%EC%A0%84%ED%8C%8C%2C_%EC%97%AD%EC%A0%84%ED%8C%8C_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# 필요한 함수들

In [ ]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    """다수의 이미지를 입력받아 2차원 배열로 변환한다(평탄화).

    Parameters
    ----------
    input_data : 4차원 배열 형태의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    col : 2차원 배열
    """
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col


def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    """(im2col과 반대) 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다.

    Parameters
    ----------
    col : 2차원 배열(입력 데이터)
    input_shape : 원래 이미지 데이터의 형상（예：(10, 1, 28, 28)）
    filter_h : 필터의 높이
    filter_w : 필터의 너비
    stride : 스트라이드
    pad : 패딩

    Returns
    -------
    img : 변환된 이미지들
    """
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]

# Affine층의 순전파와 역전파

범용적인 affine 층 구현

In [ ]:
# 4차원 텐서를 염두해둔 코드이기 떄문에 다음 학기때완전히 이해가능
class Affine:
    def __init__(self, W, b):
        self.W = W # 가중치 행렬
        self.b = b # bias 벡터

        # 입력된 data 초기화
        self.x = None
        self.original_x_shape = None
        # 가중치와 bias 미분
        self.dW = None
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1) # 4차원 텐서를 행렬로 변환. -1을 입력하면 reshape 가능한 숫자를 알아서 대입한다.
        self.x = x

        out = np.dot(self.x, self.W) + self.b # affine 변환. bias의 경우 배치처리를 할경우 행렬뎃섬에서 shape이 맞지 않는데 numpy에서 자동으로 벡터를 복사해 각 열에 더하게 된다.

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 행렬의 경우 axis 0: 행, 1: 열. 텐서의 경우 axis 0: 각 행렬, 1: 각 행렬의 행, 2: 각 행렬의 열

        dx =dx.reshape(*self.original_x_shape) # 입력데이터

        return dx

In [ ]:
x = np.array([[1,1]])

W = np.array([[1,2,3],[4,5,6]])

b = np.array([7,8,9])

test = Affine(W,b)

In [ ]:
# 순전파 테스트
test.forward(x)

In [ ]:
# 역전파 테스트
print(test.backward(np.array([[2,1,-1]])))
print(test.dW)
print(test.db)

In [ ]:
# 배기처리 테스트
y = np.array([[1,1],[2,3]])

# 순전파 테스트
print(test.forward(y))

# 역전파 테스트
print(test.backward(np.array([[2,1,-1],[1,1,1]])))
print(test.dW)
print(test.db)

간단한 affine층 구현

In [ ]:
class Affine_t:
    def __init__(self, W, b): # W:가중치 matrix, b:bias
        self.W = W
        self.b = b

    def forward(self, x):
        self.x = x # x:data
        out = np.dot(self.x, self.W) + self.b # affine 변환

        return out

    def backward(self, dout): # dout:흘러들어온 미분
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0) # 덧셈노드의 역전파는 원래 흘러들어온 값이 그대로 통과하지만 배치처리를 할경우 계산을 위해 편향벡터를 배치크기만큼 반복한 행이 만들어져 행렬이 구성된다.
                                       # 이 경우 각열을 모두 더한 값이 역전파의 결과로 반환된다.(수학적인 증명은 아님.) 자세한 설명은 노트참고

        return dx

softmax 함수

In [ ]:
def softmax(x):
    if x.dim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

cross entropy 함수

In [ ]:
def corss_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

# Sigmoid층의 순전파와 역전파

sigmoid함수는 고전적인 미분값계산시 미분계산이 간단한 덧셈, 곱셈 문제로 바뀐다. 또한 계산그래프로도 이러한 결과를 도출할 수 있다.(노트 참고)

sigmoid함수의 역전파는

$\frac{\partial L}{\partial y}y(1-y)$

In [ ]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

# RELU층의 순전파와 역전파

RELU함수는 Deep Nueral Network에서 발생하는 sigmoid함수의 vanishing gradient문제를 해결할 수 있다.(노트 참고)

In [ ]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0) # self.mask는 0보다 크면 false 0보다 작으면 true인 리스트를 반환한다.
        out = x.copy()
        out[self.mask] = 0 # 설명1

        return out

    def backward(self, dout):
        dout[self.mask] = 0 # 위의 설명1을 참조. 주의할점은 흘러들어온 미분값의 부호에 의해 1이나 0을 곱하는것이 정해지는게 아니라 위의 x값을 기준으로 gradient를 살리거나 죽이게 된다.
        dx = dout

        return dx

테스트

In [ ]:
BTS = Relu()

print(BTS.forward(np.array([1,-2,3,-4])))
print(BTS.mask)
print(BTS.backward(np.array([1,2,-3,-4])))

설명

In [ ]:
# 설명1
test = np.array([1,2,3,4,])
test1 = np.array([True, False, False, False])
test[test1] = 0
test

# Softmax with Loss층의 순전파와 역전파
softmax층과 loss층은 특이하게도 따로 미분하는 것 보다 합성합수로 미분하는 것이 더 간단한 식을 도출할 수 있다. 고전적인 미분이나 계산 graph를 이용한 역전파로 구할 수 있는 미분계수는 뺄셈연산으로 간단하게 변한다.

$\frac{\partial L}{\partial a} = y-t$

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실함수
        self.y = None # softmax의 출력(확률 벡터)
        self.t = None # 정답 레이블(one-hot encoding 형태)

    def forward(self, x,  t):
        self.t = t
        self.y = softmax(x)
        self.loss = corss_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout = 1): # 역전파의 시작은 1 이다.
        batch_size = self.t.shape[0]

        if self.t.size == self.y.size: # 정답 레이블이 원-핫 인코딩이 돼어 있는 경우.
            dx = (self.y - self.t) / batch_size # 손실함수의 정의 값은 각각의 손실함수값의 평균이므로 배치사이즈로 나누어 준다.

        else: # 원-핫 이코딩이 돼어 있지 않은 경우.
            dx = self.y.copy()
            dx[np.arange(batch_size), self.t] -= 1 # 도출된 확률벡터는 배치처리로 묶여있다는 것을 생각해보면 위와 같은 결과를 도출함.
            dx = dx / batch_size

            return dx

# BatchNormalization

BatchNormalization은 각 행이 데이터인 배치 행렬의 각 열(data의 각 feature)을 평균이 0이고 표준편차가 1인 정규분포로 변환한 후 머신이 최적의 scailing:$\gamma$ 와 shift:$\beta$를 찾아 적용하게 한다. 최종적으로 배치처리된 data의 각 featur의 표준편차가 $\gamma$ 평균이 $\beta$인 분포를 가지게 된다.

In [ ]:
class BatchNormalization:

    def __init__(self, gamma, beta, momentum = 0.9, running_mean = None, running_var = None): # gamma와 beta는 머신이 학습해 최적값을 찾지만 초기값을 설정해 줘야 한다.
                                                                                              # running_mean = None, running_var = None : 학습중엔 각 배치행렬(data 묶음)의 열(feature)을 사용해
                                                                                              # 평균과, 분산을 구해 사용하지만 학습이 끝난후엔 학습하는 동안의 전체 훈련용 data의 평균과 분산(비슷한 값?)
                                                                                              # 을 사용해 normalizetion 한다.
        self.gamma = gamma
        self.beta = beta
        self.momentum = momentum
        self.input_shape = None # 합성곱 계층은 4차원, 완전연결 계층은 2차원

        # test용 평균, 분산
        self.running_mean = running_mean
        self.running_var = running_var

        # backward 시에 사용할 중간 데이터
        self.batch_size = None
        self.xc = None
        self.std = None
        self.dgamma = None
        self.dbeta = None

    def forward(self, x, train_flg = True): # x는 입력 data, train_flag : 위의 주석처럼 훈련시와 test시의 순전파방식이 다르기 때문에 훈련시 train_flg를 True로 지정해 이를 구분한다.
        self.input_shape = x.shape
        if x.ndim != 2: # 행렬이 아니면. 지금까지는 data를 flatten한후 배치처리 했기때문에 인풋이 행렬이었지만, cnn등에서는 data를 faltten시키지 않기때문에 행렬이 아니라 텐서가 입력된다.
                        # N은 배치사이즈, C는 채널, H는 세로해상도, W는 가로해상도
            N, C, H, W = x.shape
            x = x.reshape(N, -1)

        out = self.__forward(x, train_flg)

        return out.reshape(*self.input_shape) # shape을 원래대로 복원한다. *는 ()를 없에준다.

    def __forward(self, x, train_flg):
        if self.running_mean is None:
            N, D = x.shape
            self.running_mean = np.zeros(D)
            self.running_var = np.zeros(D)

        if train_flg: # 훈련시.
            mu = x.mean(axis = 0) # 열의 평균
            xc = x - mu # 각 열의 평균이 0이 됨.
            var = np.mean(xc**2, axis = 0) # 분산
            std = np.sqrt(var + 10e-7) # 표준편차
            xn = xc / std # 각 열의 표준편차가 1이됨.

            self.batch_size = x.shape[0]
            self.xc = xc
            self.xn = xn
            self.std = std
            self.running_mean = self.momentum * self.running_mean + (1 - self.momentum) * mu # RMSProp에서 사용한 기법처럼 평균벡터를 계속해서 내분하는 과정. 결과적으로 계속해서 바뀌는 배치묶음의 평균 추세를 따라간다.
            self.running_var = self.momentum * self.running_var + (1 - self.momentum) * var

        else: # train_flg가 false일 경우 즉, test시에는 running_mean와 running_var을 사용한다.
            xc = x - self.running_mean
            xn = xc / ((np.sqrt(self.running_var + 10e-7)))

        out = self.gamma * xn + self.beta # gamma를 곱해 표준편차를 조정하고 beta를 더해 평균을 조정한다.

        return out

    def backward(self, dout):
        if dout.ndim != 2:
            N, C, H, W = dout.shape
            dout = dout.reshape(N, -1)

        dx = self.__backward(dout)

        dx = dx.reshape(*self.input_shape)

        return dx

    def __backward(self, dout):
        dbeta = dout.sum(axis = 0) # 덧셈노드를  브로드캐스팅(repeat 노드를 통과)방식으로 계산하기때문에 덧셈노드의 역전파뿐 아니라 repeat노드의 역전파인 sum노드를 추가해야한다.
        dgamma = np.sum(self.xn * dout, axis = 0) # *는 adamard  product임.
        dxn = self.gamma * dout
        dxc = dxn / self.std
        dstd = -np.sum((dxn * self.xc) / (self.std * self.std), axis = 0)
        dvar = 0.5 * dstd / self.std
        dxc += (2.0 / self.batch_size) * self.xc * dvar
        dmu = np.sum(dxc, axis = 0)
        dx = dxc - dmu / self.batch_size

        self.dgamma = dgamma
        self.dbeta = dbeta

        return dx

test, 검산

In [ ]:
gamma = np.array([np.sqrt(2), 2 * np.sqrt(2), np.sqrt(30)])
beta = np.array([2, 4, 5])

ico = BatchNormalization(gamma, beta)

x = np.array([[1, -1, 1], [2, -2, 1], [3, -3, 2], [4, -4, 2], [5, -5, 4]])

In [ ]:
ico.forward(x)

In [ ]:
dout = np.array([[1, -1, 4], [-1, 1, -4], [1, -1, 4], [-1, 1, -4], [1, -1 ,4]])

In [ ]:
ico.backward(dout)

In [ ]:
ico.dbeta

In [ ]:
ico.dgamma

# Dropout

뉴런을 무작위로 선택해 삭제하여 신호전달을 차단한다.

In [ ]:
class Dropout:

    def __init__(self, dropout_ratio = 0.5):
        self.dropout_ratio = dropout_ratio
        self.mask = None

    def forward(self, x, train_flg = True): # Dropout층은 훈련시에만 적용함.
        if train_flg:
            self.mask = np.random.rand(*x.shape) > self.dropout_ratio # .rand는 0부터 1까지의 균등분포로 랜덤값을 생성함. self.mask는 true, false로 이루어진 행렬이 됨.
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio) # 신호량을 맞추기위해 리스케일링. 이해안감.

    def backward(self, dout):
        return dout * self.mask # 신호를 죽인부분은 미분도 죽임.

# Convolution

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad

        # 중간 데이터（backward 시 사용）
        self.x = None
        self.col = None
        self.col_W = None

        # 가중치와 편향 매개변수의 기울기
        self.dW = None
        self.db = None

    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout): # convolution층은 im2col -> affine -> reshape층으로 세분화해 생각할 수 있다.
        FN, C, FH, FW = self.W.shape

        # reshape층의 역전파 재배열의 역전파는 역재배열이다. 최종적으로 행렬이 반환된다.
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        # affine층의 역전파. (가중치에 대한 미분)
        self.db = np.sum(dout, axis=0) # 각 열을 합하는 이유는 파이썬의 브로드캐스팅에 의해 편향을 더할때 리피트노드가 숨어있기 때문이다. (리피트의 역전파는 sum)
        self.dW = np.dot(self.col.T, dout)

        # 필터를 가중치행렬로 만드는 과정의 역전파.
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        # affine층 역전파. (데이터에 대한 미분)
        dcol = np.dot(dout, self.col_W.T)
        # im2col층의 역전파
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx

# Max Pooling

Max pooling층은 개념은 간단하지만 연산속도등의 이유로 실제 구현시에는 im2col을 사용해 연산하므로 코드가 복잡해진다.

1. 데이터에 im2col을 취한다.
2. 열이 PH*PW가 되도록 reshape을 한다.
3. 각 행에 최대를 취한다.
4. N*OH*OW*C로 reshape을 한다.
5. transpose(0,3,1,2)를 취한다.

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad

        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # im2col
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        # 열이 PH*PW가 되도록 reshpae
        col = col.reshape(-1, self.pool_h*self.pool_w)
        # 역전파시 필요한 과정
        arg_max = np.argmax(col, axis=1)
        # 각행의 최대값을 추출
        out = np.max(col, axis=1)
        # NOHOW*C로 reshape -> transpose(0,3,1,2)를 취한다.
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        self.x = x
        self.arg_max = arg_max

        return out

    def backward(self, dout):
        dout = dout.transpose(0, 2, 3, 1)

        pool_size = self.pool_h * self.pool_w
        dmax = np.zeros((dout.size, pool_size))
        dmax[np.arange(self.arg_max.size), self.arg_max.flatten()] = dout.flatten()
        dmax = dmax.reshape(dout.shape + (pool_size,))

        dcol = dmax.reshape(dmax.shape[0] * dmax.shape[1] * dmax.shape[2], -1)
        dx = col2im(dcol, self.x.shape, self.pool_h, self.pool_w, self.stride, self.pad)

        return dx